# lab 5: translation simple ecoder-decocer over the b2 dataset

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
import pandas as pd
import unicodedata
import string
import re
import random
import copy
from contra_qa.plots.functions  import simple_step_plot
import  matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from nltk.translate.bleu_score import sentence_bleu


% matplotlib inline

### Preparing data

In [2]:
df2 = pd.read_csv("data/boolean2_train.csv")
df2_test = pd.read_csv("data/boolean2_test.csv")

df2["text"] = df2["sentence1"] + df2["sentence2"] 
df2_test["text"] = df2_test["sentence1"] + df2_test["sentence2"] 

all_sentences = list(df2.text.values) + list(df2_test.text.values)

df2train = df2.iloc[:8500]
df2valid = df2.iloc[8500:]

In [3]:
df2train.tail()

,sentence1,sentence2,and_A,and_B,label,text
8495,Agnes is proud and Lauren is ambitious,Agnes is not proud,Agnes is proud,Lauren is ambitious,1,Agnes is proud and Lauren is ambitious Agnes i...
8496,Curtis is wonderful and Jessie is ambitious,Curtis is not wonderful,Curtis is wonderful,Jessie is ambitious,1,Curtis is wonderful and Jessie is ambitious Cu...
8497,Brett is dead and Tracy is important,Tracy is not important,Brett is dead,Tracy is important,1,Brett is dead and Tracy is important Tracy is ...
8498,Lauren is hallowed and Yvette is shy,Yvette is not shy,Lauren is hallowed,Yvette is shy,1,Lauren is hallowed and Yvette is shy Yvette is...
8499,Kathleen is alive and Dustin is clever,Dustin is not clever,Kathleen is alive,Dustin is clever,1,Kathleen is alive and Dustin is clever Dustin ...


In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


example = "ddddda'''~~çãpoeéééééÈ'''#$$##@!@!@AAS@#12323fdf"
print("Before:", example)
print()
print("After:", normalizeString(example))

Before: ddddda'''~~çãpoeéééééÈ'''#$$##@!@!@AAS@#12323fdf

After: ddddda capoeeeeeee ! ! aas fdf


In [6]:
pairs_A = list(zip(list(df2train.sentence1.values), list(df2train.and_A.values)))
pairs_B = list(zip(list(df2train.sentence1.values), list(df2train.and_B.values)))
pairs_A = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_A]
pairs_B = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_B]
pairs_A_val = list(zip(list(df2valid.sentence1.values), list(df2valid.and_A.values)))
pairs_B_val = list(zip(list(df2valid.sentence1.values), list(df2valid.and_B.values)))
pairs_A_val = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_A_val]
pairs_B_val = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_B_val]


In [7]:
all_text_pairs = zip(all_sentences, all_sentences)
all_text_pairs = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in all_text_pairs]

In [8]:
def readLangs(lang1, lang2, pairs, reverse=False):
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [tuple(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [9]:
MAX_LENGTH = 20


def filterPair(p):
    cond1 = len(p[0].split(' ')) < MAX_LENGTH
    cond2 = len(p[1].split(' ')) < MAX_LENGTH 
    return cond1 and cond2


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


In [10]:
def prepareData(lang1, lang2, pairs, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, pairs, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [11]:
_, _, training_pairs_A = prepareData("eng_enc",
                                             "eng_dec",
                                             pairs_A)

print()


input_lang, _, _ = prepareData("eng_enc",
                               "eng_dec",
                               all_text_pairs)

output_lang = copy.deepcopy(input_lang)

print()

print()
_, _, valid_pairs_A = prepareData("eng_enc",
                                "eng_dec",
                                pairs_A_val)

Read 8500 sentence pairs
Trimmed to 8500 sentence pairs
Counting words...
Counted words:
eng_enc 705
eng_dec 704

Read 11000 sentence pairs
Trimmed to 11000 sentence pairs
Counting words...
Counted words:
eng_enc 706
eng_dec 706


Read 1500 sentence pairs
Trimmed to 1500 sentence pairs
Counting words...
Counted words:
eng_enc 696
eng_dec 644


In [12]:
_, _, training_pairs_B = prepareData("eng_enc",
                                     "eng_dec",
                                     pairs_B)
print()
_, _, valid_pairs_B = prepareData("eng_enc",
                                "eng_dec",
                                pairs_B_val)

Read 8500 sentence pairs
Trimmed to 8500 sentence pairs
Counting words...
Counted words:
eng_enc 705
eng_dec 704

Read 1500 sentence pairs
Trimmed to 1500 sentence pairs
Counting words...
Counted words:
eng_enc 696
eng_dec 638


### sentences 2 tensors

In [13]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

In [14]:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [15]:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

### models

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [17]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
hidden_size = 100
eng_enc_v_size = input_lang.n_words
eng_dec_v_size = output_lang.n_words

In [19]:
encoderA = EncoderRNN(eng_enc_v_size, hidden_size)
decoderA = DecoderRNN(hidden_size, eng_dec_v_size)
encoderA.load_state_dict(torch.load("encoder4.pkl"))
decoderA.load_state_dict(torch.load("decoder4.pkl"))

In [20]:
encoderB = EncoderRNN(eng_enc_v_size, hidden_size)
decoderB = DecoderRNN(hidden_size, eng_dec_v_size)
encoderB.load_state_dict(torch.load("encoder5.pkl"))
decoderB.load_state_dict(torch.load("decoder5.pkl"))

## translating

In [21]:
def translate(encoder,
              decoder,
              sentence,
              max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(
            max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, topone = decoder_output.data.topk(1)
            if topone.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topone.item()])

            decoder_input = topone.squeeze().detach()

        return " ".join(decoded_words)

## translation of a trained model: and A

In [22]:
for t in training_pairs_A[0:3]:
    print("input_sentence : " + t[0])
    neural_translation = translate(encoderA,
                                   decoderA,
                                   t[0],
                                   max_length=MAX_LENGTH)
    print("neural translation : " + neural_translation)
    reference = t[1] + ' <EOS>'
    print("reference translation : " + reference)
    reference = reference.split(" ")
    candidate = neural_translation.split(" ")
    score = sentence_bleu([reference], candidate)
    print("blue score = {:.2f}".format(score))
    print()

input_sentence : penny is thankful and naomi is alive
neural translation : blanche is thankful <EOS>
reference translation : penny is thankful <EOS>
blue score = 0.71

input_sentence : carlos is kind and paula is uninterested
neural translation : carlos is kind <EOS>
reference translation : carlos is kind <EOS>
blue score = 1.00

input_sentence : jack is hallowed and kent is easy
neural translation : jack is hallowed <EOS>
reference translation : jack is hallowed <EOS>
blue score = 1.00



/usr/local/lib/python3.5/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


## translation of a trained model: and B

In [23]:
for t in training_pairs_B[0:3]:
    print("input_sentence : " + t[0])
    neural_translation = translate(encoderB,
                                   decoderB,
                                   t[0],
                                   max_length=MAX_LENGTH)
    print("neural translation : " + neural_translation)
    reference = t[1] + ' <EOS>'
    print("reference translation : " + reference)
    reference = reference.split(" ")
    candidate = neural_translation.split(" ")
    score = sentence_bleu([reference], candidate)
    print("blue score = {:.2f}".format(score))
    print()

input_sentence : penny is thankful and naomi is alive
neural translation : naomi is alive <EOS>
reference translation : naomi is alive <EOS>
blue score = 1.00

input_sentence : carlos is kind and paula is uninterested
neural translation : paula is uninterested <EOS>
reference translation : paula is uninterested <EOS>
blue score = 1.00

input_sentence : jack is hallowed and kent is easy
neural translation : kent is easy <EOS>
reference translation : kent is easy <EOS>
blue score = 1.00



## Defining meta model

In [24]:
class AndModel(nn.Module):
    def __init__(self,
                 encoderA,
                 decoderA,
                 encoderB,
                 decoderB,
                 hidden_size,
                 output_size,
                 max_length,
                 input_lang,
                 target_lang,
                 SOS_token=0,
                 EOS_token=1):
        super(AndModel, self).__init__()
        self.max_length = max_length
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.encoderA = encoderA
        self.decoderA = decoderA
        self.encoderB = encoderB
        self.decoderB = decoderB
        self.input_lang = input_lang
        self.target_lang = target_lang
        self.SOS_token = SOS_token
        self.EOS_token = EOS_token
        
    
    def encode(self,
               sentence,
               encoder,
               is_tensor,
               hidden=None):
        if not is_tensor:
            input_tensor = tensorFromSentence(self.input_lang, sentence)
        else:
            input_tensor = sentence

        input_length = input_tensor.size()[0]
        
        if hidden is None:
            encoder_hidden = encoder.initHidden()
        else:
            encoder_hidden = hidden
            
        
        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei],
                                        encoder_hidden)
        return encoder_hidden
    
    
    def decode(self,
               tensor,
               decoder,
               out_tensor):
        
        decoder_input = torch.tensor([[self.SOS_token]], device=device)
        decoder_hidden = tensor
        decoded_words = []

        for di in range(self.max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            _, topone = decoder_output.data.topk(1)
            if topone.item() == self.EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(self.target_lang.index2word[topone.item()])

            decoder_input = topone.squeeze().detach()
        
        if not out_tensor:
            output = " ".join(decoded_words)
        else:
            output = decoder_hidden

        return output
    
    def sen2vec(self, sentence, encoder, decoder, is_tensor, out_tensor):
        encoded = self.encode(sentence, encoder, is_tensor)
        vec = self.decode(encoded, decoder, out_tensor)
        return vec
    
    def sen2vecA(self, sentence, is_tensor):
        encoded = self.encode(sentence, self.encoderA, is_tensor)
        vec = self.decode(encoded, self.decoderA, out_tensor=True)
        return vec
    
    def sen2vecB(self, sentence, is_tensor):
        encoded = self.encode(sentence, self.encoderB, is_tensor)
        vec = self.decode(encoded, self.decoderB, out_tensor=True)
        return vec
    
    def forward(self, s1, s2, is_tensor=True):
        pass
        


In [25]:
a = AndModel(encoderA,
             decoderA,
             encoderB,
             decoderB,
             hidden_size=100,
             output_size=2,
             max_length=MAX_LENGTH,
             input_lang=input_lang,
             target_lang=output_lang)

Test encoding decoding

In [26]:
for ex in training_pairs_B[0:3]:
    print("===========")
    ex = ex[0]
    print("s1:\n")
    print(ex)
    print()

    
    ex_A = a.sen2vec(ex,
                     a.encoderA,
                     a.decoderA,
                     is_tensor=False,
                     out_tensor=False)
    
    ex_B = a.sen2vec(ex,
                     a.encoderB,
                     a.decoderB,
                     is_tensor=False,
                     out_tensor=False)

    print("inference A:\n")
    print(ex_A)
    print()
    print("inference B:\n")
    print(ex_B)

s1:

penny is thankful and naomi is alive

inference A:

blanche is thankful <EOS>

inference B:

naomi is alive <EOS>
s1:

carlos is kind and paula is uninterested

inference A:

carlos is kind <EOS>

inference B:

paula is uninterested <EOS>
s1:

jack is hallowed and kent is easy

inference A:

jack is hallowed <EOS>

inference B:

kent is easy <EOS>


In [27]:
for ex in training_pairs_B[0:1]:
    print("===========")
    ex = ex[0]
    print("s1:\n")
    print(ex)
    print()

    
    ex_A = a.sen2vec(ex,
                     a.encoderA,
                     a.decoderA,
                     is_tensor=False,
                     out_tensor=False)
    ex_A = a.sen2vecA(ex,is_tensor=False)
    ex_B = a.sen2vecB(ex,is_tensor=False)
    
    print(ex_A)
    print()
    print(ex_B)

s1:

penny is thankful and naomi is alive

tensor([[[-0.4259, -0.5321,  0.6835,  0.9980,  0.9865, -0.0326, -0.8547,
          -0.6571,  0.5157,  0.0794,  0.4141, -0.3890,  1.0000, -0.9051,
           0.9944,  0.9096,  0.0875, -0.9996,  0.9112, -0.0006, -0.9984,
          -0.7040, -0.0880, -0.7222, -0.9990,  0.7732, -0.9064,  0.2055,
           0.1898,  0.8505, -0.6891, -0.9391,  0.9999, -0.9916,  0.5694,
           0.3941,  0.8801,  0.9980, -0.5498, -0.8040, -0.1095, -0.2328,
          -0.9724,  0.8991, -0.2609, -0.0054,  0.9327, -0.9945, -0.1388,
          -0.9704, -0.9887,  0.9991,  0.8406,  0.0321, -0.9388, -0.6922,
           0.0622, -0.9972, -0.9954,  0.8046, -0.4433,  0.9741, -0.8358,
          -0.9171, -0.9966, -0.4546,  0.9799, -0.1956,  0.9755, -0.5717,
          -0.6437, -0.4731, -0.7418,  0.9960, -0.9396,  0.0081,  0.4872,
          -0.9808, -0.2468,  0.7522,  0.2154,  0.9563,  0.9638,  0.5954,
           0.8221, -0.0410,  0.8202, -0.9973,  0.4164, -0.1902, -0.9980,
        